In [1]:
import pandas as pd
import datetime as dt
from dateutil.relativedelta import relativedelta
from dateutil.relativedelta import MO
import numpy as np

import filepath
import schedule_methods
import smartsheet
import os
from pathlib import Path

from gsheets import Sheets
import pygsheets

import send_email
import infrastructure

In [2]:
#testing pipeline
sms_test = infrastructure.get_smartsheet('test_sms')

In [3]:
#Publisher Config
gc = pygsheets.authorize(service_account_file=filepath.service_account_location)
publisher_config = gc.open_by_url('https://docs.google.com/spreadsheets/d/1Tzda6Djr3zQmOhWu7Ief3GVR9Cjaml8238CeX7chj_U/edit#gid=1620368362') 
pub_config = publisher_config[0].get_as_df()
pub_config.rename(columns={'DP.DS or DP.sV': 'DP.SV'}, inplace=True)

/Users/nathan/opt/anaconda3/lib/python3.8/site-packages/pygsheets/worksheet.py:1554: UserWarning: At least one column name in the data frame is an empty string. If this is a concern, please specify include_tailing_empty=False and/or ensure that each column containing data has a name.
  warnings.warn('At least one column name in the data frame is an empty string. If this is a concern, please specify include_tailing_empty=False and/or ensure that each column containing data has a name.')


In [4]:
#la nina
gc = pygsheets.authorize(service_account_file=filepath.service_account_location)
lanina = gc.open_by_url('https://docs.google.com/spreadsheets/d/1obszkCQoE0ELOR1O0CrLVETUEmEIWlGuyAmK3FgWSJg/edit#gid=1099746391') 
lanina_wks  = lanina[1].get_as_df()

In [5]:
def active_account(df): 
    most_recent_date = df.groupby('DP&Pub')['Date'].max().reset_index()
    most_recent_date['last_2_date'] = pd.Timestamp(df['Date'].max()-dt.timedelta(days=2)) 
    active_pubid = most_recent_date.loc[most_recent_date['Date'] > most_recent_date['last_2_date'],]
    return active_pubid
   
def new_affiliateID(df):
    # This function is to create a new Affiliate ID to differentiate SC.RF & SC.FHA, LPG.FHA & LPG.RF 
    df['DP&Pub'] = df['DP&Pub'].str.upper().replace(PubDP_to_DPPub)
    df['Affliate ID'] = df['DP&Pub'].str[-6:]
    df['new Affliate ID'] = df['Affliate ID']
    return df

def payout_transform(df):
    # extract payout as a number 
    df['Payout_rate'] = df['$ Payout'].astype('str').str.extract(pat = '([0-9\.]+)',expand = True)
    df['Payout_rate'] = pd.to_numeric(df['Payout_rate'], errors='coerce')
    df = df.dropna(subset=(['Payout_rate']))
    df['Payout_rate'] = df['Payout_rate'].astype('float')
    df.loc[df['Payout_rate'].isna() == True,'Payout Type' ] = 'variable'
    df.loc[df['Payout Type'] == 'REV SHARE','Payout Type' ] = 'variable'
    df.loc[df['Payout Type'] == 'variable','Payout_rate' ] = np.nan
    return df

In [6]:
# DATA MANIPULATION #
#offers sheet
sms_offer = infrastructure.get_smartsheet('offers_sms')
sms_offer = sms_offer[sms_offer['Hitpath Offer ID'].isna() == False]
sms_offer['Hitpath Offer ID'] = sms_offer['Hitpath Offer ID'].astype('int').astype('str')
sms_offer = payout_transform(sms_offer)

#mamba
mamba = infrastructure.get_mamba()

#lc_merged_data
df = pd.read_csv('SS_LC_merged_data.csv')
df = infrastructure.transform_sms_df(df)
df['Date'] = pd.to_datetime(df['Date'])
df.rename(columns={'Hitpath_Offer_ID': 'Hitpath Offer ID'}, inplace=True)
df['Hitpath Offer ID'] = pd.to_numeric(df['Hitpath Offer ID'], errors='coerce')
df = df.dropna(subset=['Hitpath Offer ID'])
df['Hitpath Offer ID'] = df['Hitpath Offer ID'].astype(int).astype(str)
df['DP&Pub']=df['DP&Pub'].str.extract("([A-Z.0-9]+_[0-9]+)", expand = True) 

<ipython-input-5-d657e1faed4f>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Payout_rate'] = df['Payout_rate'].astype('float')
/Users/nathan/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/Users/nathan/opt/anaconda3/lib/python3.8/site-packages/pygsheets/worksheet.py:1554: UserWarning: At least one column name in the data frame is an empty string. If this is a concern, please specify inc

In [7]:
df.columns

Index(['Date', 'Scheduling Time', 'Offer', 'Hitpath Offer ID', 'DP.SV',
       'Affiliate ID', 'DP&Pub', 'Job_Id', 'Job_Name', 'Creative_Id',
       'Creative Type', 'Creative', 'Send Strategy', 'Shortcode',
       'Start_Tstamp', 'Sent', 'Segments', 'Revenue', 'Conversions',
       'Jump Page Clicks', 'Delivered', 'Not_Delivered', 'Optout', 'Clicks',
       'Cost', 'Revenue CPM (eCPM)', 'Time', 'Publisher', 'Campaign', 'Route',
       'Carrier', 'Dataset', 'Message', 'Responder Template', 'Keyword',
       'Responder', 'Router Domain Name', 'c1', 'Responded', 'Response Rate',
       'CTR', 'Gross Profit', 'Margin', 'RPU', 'Provider', 'Code_Type',
       'Revenue Source', 'Ar Day', 'Sub_Id', 'Campaign_Id', 'Roi',
       'Shortcode Name', 'Total', 'Jump Page Version', 'Offer Wall Id',
       'Offer Type', 'campaign_id', 'campaign_name', 'shortcode_DP.SV',
       'Opportunity Cost Send Strategy', 'Dataset_Agg_30D_eCPM',
       'Opportunity Cost', 'Offer Vertical', 'Offer Gap', 'Vertical 

In [8]:
sms_offer

,Hitpath Offer ID,Email Hitpath Offer ID,Scheduling Name,Offer Name,Status,Status Changed Date,Delayed Reporting,RX Rep,Advertiser,Data Partner Relationship,...,Budget Status,Budget,Cap,Portal Acct Info,Live Pub IDs,Paused Pubs,Special Pub Payouts,Pitch Approval,Single/Multiple Pitch,Payout_rate
4,13152,12980.0,13152-Credit Card-3 West Media-Indigo Masterca...,Indigo Mastercard CPL v3 (non-rewards),Set Up: Pending Custom Content,03/25/24,NaN,Sarah Bowe,3 West Media,NaN,...,Non Budgeted,NaN,NaN,3 West Media(ID:28528) - Haley - SMS,NaN,NaN,NaN,NaN,Single,52.0
5,12920,11206.0,12920-Credit Card-3 West Media-Milestone Maste...,Milestone Mastercard - Cashback Rewards CPL (NF),Set Up: Pending Custom Content,01/24/24,NaN,Sarah Bowe,3 West Media,NaN,...,NaN,NaN,NaN,28528,NaN,NaN,NaN,Approved,Single,52.0
6,12919,10914.0,12919-Credit Card-3 West Media-Destiny Masterc...,Destiny Mastercard - Cashback Rewards CPL (NF),Set Up: Pending Custom Content,01/24/24,NaN,Sarah Bowe,3 West Media,NaN,...,NaN,NaN,NaN,28528,NaN,NaN,NaN,Approved,Single,52.0
7,12917,12396.0,12917-Credit Card-3 West Media-Destiny Masterc...,Destiny Mastercard - Instant Credit CPL,Set Up: Pending Custom Content,01/24/24,NaN,Sarah Bowe,3 West Media,NaN,...,NaN,NaN,NaN,28528,NaN,NaN,NaN,Approved,Single,50.0
8,12916,NaN,12916-Auto Warranty-3 West Media-Road Master A...,Road Master Auto Warranty CPL,Set Up: Pending Custom Content,01/24/24,NaN,Sarah Bowe,3 West Media,NaN,...,NaN,NaN,NaN,28528,NaN,NaN,NaN,Approved,Single,16.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
480,10710,NaN,10710-Sweepstakes-Yep Ads-TechDealGiveaway - $...,TechDealGiveaway - $750 Venmo Gift Card,Paused: Internal,NaN,NaN,Rhiannon Selander,Yep Ads,NaN,...,NaN,NaN,No Cap,NaN,NaN,NaN,NaN,NaN,NaN,25.0
481,12169,NaN,12169-Gift Card-Yep Ads-RiotRewards - Chevron ...,RiotRewards - Chevron $500 Fuel Gift Card (Trial),Paused: Internal,NaN,NaN,Rhiannon Selander,Yep Ads,NaN,...,NaN,100,Daily,Dustin - SMS,NaN,NaN,NaN,NaN,NaN,25.0
482,12175,NaN,12175-Gift Card-Yep Ads-RiotRewards - BP $500 ...,RiotRewards - BP $500 Fuel Gift Card (Trial),Paused: Internal,NaN,NaN,Rhiannon Selander,Yep Ads,NaN,...,NaN,100,Daily,Dustin - SMS,NaN,NaN,NaN,NaN,NaN,25.0
483,12194,NaN,12194-Sweepstakes-Yep Ads-RiotRewards - $500 W...,RiotRewards - $500 Walmart Gift Card,Paused: Internal,NaN,NaN,Rhiannon Selander,Yep Ads,NaN,...,NaN,100,Daily,Dustin - SMS,NaN,NaN,NaN,NaN,NaN,25.0


In [9]:
offer_id_and_vertical = sms_offer[['Hitpath Offer ID', 'Vertical','Payout_rate', 'Payout Type']]

# Convert types to ensure proper matching
offer_id_and_vertical['Hitpath Offer ID'] = offer_id_and_vertical['Hitpath Offer ID'].astype(str)

# Merge to add the 'Vertical' column to the sample data based on matching Hitpath Offer ID
df = pd.merge(df, offer_id_and_vertical, on='Hitpath Offer ID', how='left')


<ipython-input-9-d559cd7ac108>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  offer_id_and_vertical['Hitpath Offer ID'] = offer_id_and_vertical['Hitpath Offer ID'].astype(str)


In [10]:
df['Conversions'] = df['Revenue']/df['Payout_rate']

In [11]:
df3 = df.copy()
df3['Hitpath Offer ID'] = df['Hitpath Offer ID'].astype('int').astype('str')
df3['DP&Pub']=df3['DP&Pub'].str.extract("([A-Z.0-9]+_[0-9]+)", expand = True) 

In [12]:
# Get current active pubid     
active_pubid = active_account(df)
active_pubid['Affiliate ID'] = active_pubid['DP&Pub'].str[-6:]
active_pubid = active_pubid.sort_values(by = "Affiliate ID")
active_pubid = active_pubid[['DP&Pub','Affiliate ID']]

In [13]:
#publisher config
pub_config['DP&Pub'] = pub_config['DP.SV'] + "_" + pub_config['PUBID'].astype('str')
pub_config['PubDP'] = pub_config['DP&Pub']
Accounts_PC = pub_config[['DP&Pub', 'PubDP']]

In [14]:
# Rewrite Affiliate ID_DP.DS to be numbers first and DP last
Accounts_PC['PubDP'] = '_' + Accounts_PC['PubDP']
Accounts_PC['ID'] = Accounts_PC['PubDP'].str.strip().str[-6:]
Accounts_PC['PubDP'] = Accounts_PC['ID'] + Accounts_PC['PubDP']
Accounts_PC['PubDP'] = Accounts_PC['PubDP'].astype(str).str[:-6]
Accounts_PC = Accounts_PC.drop(columns=['ID'])
Accounts_PC['PubDP'] = Accounts_PC['PubDP'].str[:-1]

# turn df into dict to map later on
DPPub_to_PubDP = dict(zip(Accounts_PC['DP&Pub'], Accounts_PC['PubDP']))
PubDP_to_DPPub = dict(zip(Accounts_PC['PubDP'], Accounts_PC['DP&Pub']))
#new pubid
pub_config=pub_config[pub_config['PUBID'].notna()]
pub_config=pub_config.drop_duplicates(subset='DP&Pub')
pub_config1 = pub_config[['DP&Pub','DMA']].drop_duplicates()
pub_config['PUBID'] = pub_config['PUBID'].astype(str).str.split(".",expand = True)[0]
pub_config['new Affiliate ID'] = pub_config['PUBID']

DMA_dict = pub_config.set_index('new Affiliate ID').to_dict()['DMA']
status_dict = pub_config.set_index('new Affiliate ID').to_dict()['INTERNAL STATUS']
pub_config=pub_config[pub_config['INTERNAL STATUS']!='INACTIVE']

<ipython-input-14-5df3d70fbff4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Accounts_PC['PubDP'] = '_' + Accounts_PC['PubDP']
<ipython-input-14-5df3d70fbff4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Accounts_PC['ID'] = Accounts_PC['PubDP'].str.strip().str[-6:]
<ipython-input-14-5df3d70fbff4>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.p

In [15]:
# offers['Budget Start Date'] = pd.to_datetime(offers['Budget Start Date'])

###filter out all the budget offers
budget_offers = sms_offer[(sms_offer["Status"].isin(["Live: Budgeted Offer","Live",'In Testing','Retest']))  & (sms_offer["Budget Status"] == "Budgeted")]
budget_offers1 = budget_offers.loc[:, ["Hitpath Offer ID",'Offer Name','Vertical',"Status","Budget Status", "Budget","Cap","$ Payout","Payout Type",'RX Rep','Advertiser']] #,'Budget Start Date']]

###Get the information we need from lexi3
df = df3.loc[:, ["Hitpath Offer ID","Date", "Segments", 'Vertical',"Delivered","Revenue","DP&Pub","Send Strategy", "Opportunity Cost", "Conversions"]] #no drop number or conversions

KeyError: "Passing list-likes to .loc or [] with any missing labels is no longer supported. The following labels were missing: Index(['Vertical'], dtype='object'). See https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike"

In [ ]:
#################### Clean up Segment ######################

###last 7 days avg delivered by each segment
last_7_day = pd.Timestamp(df['Date'].max()-dt.timedelta(days=6)) 
last_7_day_df = df.loc[df['Date']>= last_7_day]
#combine
df1= last_7_day_df.groupby(['DP&Pub','Date','Segments'])['Delivered'].sum().reset_index()
df1 = new_affiliateID(df1)
avgDelivered= df1.groupby(['new Affliate ID','Segments'])['Delivered'].median().reset_index()

In [ ]:
####################  Average Conversions of last 3 drops for budget offers by accounts ######################
# get DF with only P drops 
df_p = df[(df["Send Strategy"] == "P")]
df_p =  new_affiliateID(df_p)

# Get number format from payout out 
budget_offers1 = payout_transform(budget_offers1)
# last 60 days lexi data with offer vertical 
budget_df = budget_offers1.merge(df_p,  how='left', on=['Hitpath Offer ID','Vertical'],copy= False)
time_period_60=pd.Timestamp(dt.date.today()-dt.timedelta(days=60))
budget_df1 = budget_df.loc[budget_df['Date']>= time_period_60]

# calculate No.conversions 
budget_df1['Conversions1'] = budget_df1['Revenue'] / budget_df1['Payout_rate']
budget_df1.loc[budget_df1['Conversions1'].isna(), 'Conversions1'] = budget_df1['Conversions']
budget_df1['Conversions'] = budget_df1['Conversions1']

# calculate average conversions based on last 3 drops
df_copy = df.merge(budget_offers1,how='left', on=['Hitpath Offer ID','Vertical'],copy= False)
df_copy['Conversions1'] = df_copy['Revenue'] / df_copy['Payout_rate']
df_copy.loc[df_copy['Conversions1'].isna(), 'Conversions1'] = df_copy['Conversions']
last_3_conversions = df_copy.groupby(['DP&Pub','Hitpath Offer ID','Segments',"Send Strategy",'Date'])['Conversions'].sum().reset_index().sort_values(by=['DP&Pub','Hitpath Offer ID','Segments','Date'])
last_3_conversions['Avg Conversions'] = last_3_conversions.groupby(['DP&Pub','Hitpath Offer ID','Segments'])['Conversions'].transform(lambda x: x.tail(3).mean())
last_3_conversions = last_3_conversions[['DP&Pub','Hitpath Offer ID','Segments','Avg Conversions']]
last_3_conversions = new_affiliateID(last_3_conversions)
last_3_conversions = last_3_conversions[['new Affliate ID','Hitpath Offer ID','Segments','Avg Conversions']].drop_duplicates()

In [ ]:
#################### Aggregate eCPM by accounts: account's offer eCPM, account's vertical eCPM, dataset eCPM ######################

# offer eCPM by accounts in the last 60 days
gpby= budget_df1.groupby(['new Affliate ID','Hitpath Offer ID'])
account_budget=gpby.agg({'Revenue':'sum','Date':'nunique'})
account_budget['avg_eCPM_budgeted_offer']=gpby.apply(lambda x:x.Revenue.sum()*1000/x.Delivered.sum())
account_budget = account_budget.reset_index()
account_budget = account_budget.rename(columns={"Date": "#Drops"})

# calculate account's vertcal eCPM in the last 30 days 
time_period_30=pd.Timestamp(dt.date.today()-dt.timedelta(days=30))
account_30_days = df_p.loc[df_p['Date']>= time_period_30]
gpby= account_30_days.groupby(['new Affliate ID','Vertical'])
account_vertical=gpby.agg({'Revenue':'sum'})
account_vertical['avg_eCPM_vertical']=gpby.apply(lambda x:x.Revenue.sum()*1000/x.Delivered.sum())
account_vertical = account_vertical.reset_index()
# calculate account's last 30 days eCPM
account_all_30_days = df_p.loc[df_p['Date']== df_p['Date'].max()]
gpby= account_30_days.groupby(['new Affliate ID'])
account=gpby.agg({'Revenue':'sum'})
account['avg_eCPM_account']=gpby.apply(lambda x:(x.Revenue.sum()-x['Opportunity Cost'].sum())*1000/x.Delivered.sum())
account = account.reset_index()

In [ ]:
#################### Estimated eCPM for each budget offers in each account  ######################
# create a pair for each budget offers and each account and assign each budget offers with a estimate eCPM. 
# If the budget offers dropped in the account in the last 60 days, we use offer's eCPM as estimate eCPM
# If the budget offers wasn't scheduled in the account before, we use account's vertical eCPM in last 30 days as estimate eCPM for the budgeted offer
# If the same vertical wans't scheduled in the account before, we use account dataset eCPM in last 30 days as estimate eCPM for the budgeted offer
new_df = budget_offers1[['Hitpath Offer ID','Vertical']].merge(pd.DataFrame(df_p['new Affliate ID'].unique()),how = 'cross')
new_df.columns = ['Hitpath Offer ID','Vertical','new Affliate ID']
budget_offers2 = new_df.merge(account_budget[['new Affliate ID','Hitpath Offer ID','#Drops','avg_eCPM_budgeted_offer']],how = 'left', on = ['Hitpath Offer ID','new Affliate ID'],copy = False)
budget_offers3 = budget_offers2.merge(account_vertical[['new Affliate ID','Vertical','avg_eCPM_vertical']],how = 'left', on =['new Affliate ID','Vertical'],copy = False)
budget_offers4 = budget_offers3.merge(account[['new Affliate ID','avg_eCPM_account']],how = 'left', on =['new Affliate ID'],copy = False)


budget_offers4['eCPM Metrics'] = "offer eCPM"
budget_offers4.loc[budget_offers4['avg_eCPM_budgeted_offer'].isna(),'eCPM Metrics'] = 'vertical eCPM'
budget_offers4.loc[budget_offers4['avg_eCPM_budgeted_offer'].isna(),'avg_eCPM_budgeted_offer'] = budget_offers4['avg_eCPM_vertical']
budget_offers4.loc[budget_offers4['avg_eCPM_budgeted_offer'].isna() ,'eCPM Metrics'] = 'account eCPM'
budget_offers4.loc[budget_offers4['avg_eCPM_budgeted_offer'].isna() ,'avg_eCPM_budgeted_offer'] = budget_offers4['avg_eCPM_account']
account_eCPM= budget_offers4[['new Affliate ID',"Hitpath Offer ID","#Drops","eCPM Metrics","avg_eCPM_budgeted_offer"]]
account_eCPM = account_eCPM.rename(columns={"avg_eCPM_budgeted_offer": "avg_eCPM"})

In [ ]:
#################### calculate Used budget for each budgeted offer ######################

budget_used = budget_offers1.merge(df, how='left', on='Hitpath Offer ID')
# get the first date of the week, the month, the year and today 
start_of_week = pd.Timestamp(dt.date.today() + relativedelta(weekday=MO(-1)))
print(start_of_week)
start_of_month = pd.Timestamp(dt.date.today() + relativedelta(day=1))
print(start_of_month)
start_of_today = pd.Timestamp(dt.date.today())
print(start_of_today)
start_of_year = pd.Timestamp(dt.date.today() +relativedelta( month=1,day = 1) )
# budget_used['Budget Start Date'] = pd.to_datetime(budget_used['Budget Start Date'])
# Based on budget's offer cap to determine the budget period. For example, for a monthly budget offer, we need to calculate the used budget started from the first date of the month. 
# budget_used = budget_used[(budget_used['Budget Start Date'].isna())| (budget_used['Date'] >= budget_used['Budget Start Date'])]
budget_past  = budget_used[((budget_used['Cap'] == 'Yearly') & (budget_used['Date'] >= start_of_year)) |
                               ((budget_used['Cap'] == 'Monthly') & (budget_used['Date'] >= start_of_month)) | 
                               ((budget_used['Cap'] == 'Weekly') & (budget_used['Date'] >= start_of_week)) |
                               ((budget_used['Cap'] == 'Daily') & (budget_used['Date'] >= start_of_today)) |
                               ((budget_used['Cap'] == 'nan') | (budget_used['Cap'] == 'Test')  | (budget_used['Cap'] == 'Lifetime') | (budget_used['Cap'] == 'PubID Lifetime') )]
# Special case: if budget cap is Pubid lifetime, we calculate budget by accounts 
budget_used_by_pubid_lifetime = budget_past.loc[budget_past['Cap'] == 'PubID Lifetime',].groupby(['DP&Pub', 'Hitpath Offer ID'])['Revenue'].sum().reset_index()
budget_used_by_pubid_lifetime.columns = ['DP&Pub', 'Hitpath Offer ID','Budget_Used_Revenue_by_pubid']

# Get revenue, no.drops and total conversions for each offer's used budget within its budget period 
budget_no_drops =  budget_past.groupby(by=['Hitpath Offer ID','Date','DP&Pub'])
budget_usedRev1=budget_no_drops.agg({'Revenue':'sum','Conversions':'sum'})
budget_usedRev1 = budget_usedRev1.reset_index().rename(columns={"Revenue":'Budget_Used_Revenue', 'Conversions':'budget_used'})
budget_usedRev = budget_usedRev1.groupby(by=['Hitpath Offer ID'])[['Budget_Used_Revenue','budget_used']].sum().reset_index()

In [ ]:
#################### calculate projected budget for each budgeted offer ######################
today = pd.Timestamp(dt.date.today())
last_date = df3['Date'].max()
# extract the upcoming schedule from cobra 
future = mamba.loc[mamba['Date'] > last_date]
# extract information for budget offers from upcoming schedule: sending date, account and segment 
future.loc[future['Segment '].isna(),'Segment '] =  future['Segment '].str.split('_').str[-1]
future['Segment '] = future['Segment '].str.strip().str.upper()
future = future[future["Hitpath Offer ID"].isna() == False]
# future['Hitpath Offer ID'] = future['Hitpath Offer ID'].astype('int').astype('str')
future_schedule = future.loc[:, ["Date", "Dataset","Hitpath Offer ID","Segment "]]
future_schedule = future_schedule[future_schedule["Hitpath Offer ID"].isna() == False]
future_schedule["Hitpath Offer ID"] = future_schedule["Hitpath Offer ID"].astype('str').str.strip()
future_schedule = future_schedule.merge(budget_offers1, how='left', on='Hitpath Offer ID')
future_schedule_budgeted = future_schedule[future_schedule["Budget Status"] == "Budgeted"]

In [ ]:
# combine budgeted offer's upcoming schedule with estimate eCPM by accounts 
future_schedule_budgeted[['Data Source', 'Affliate ID']] = future_schedule_budgeted['Dataset'].str.rsplit('_', n=1, expand=True)
future_schedule_budgeted['DP&Pub'] = future_schedule_budgeted[['Data Source', 'Affliate ID']].agg('_'.join, axis=1)
account_eCPM = account_eCPM[['Hitpath Offer ID', 'avg_eCPM','new Affliate ID','#Drops', 'eCPM Metrics']]
account_eCPM = account_eCPM.loc[account_eCPM['avg_eCPM'].isna() == False,]
future_schedule_budgeted = new_affiliateID(future_schedule_budgeted)
future_schedule_eCPM = future_schedule_budgeted.merge(account_eCPM,how='left',on=['new Affliate ID', 'Hitpath Offer ID'], copy = False)
#fix some segment
# future_schedule_eCPM['real seg'] = future_schedule_eCPM['real seg'].apply(fix_seg)

#merge average delivered to future schedule
avgDelivered = avgDelivered[['Segments', 'Delivered','new Affliate ID']]
future_deli= future_schedule_eCPM.merge(avgDelivered, how='left', left_on=['new Affliate ID', 'Segment '], right_on=['new Affliate ID', 'Segments'],copy = False)
#merge last 3 drops's avg conversions to future schedule
future_deli= future_deli.merge(last_3_conversions, how='left', left_on=['new Affliate ID', 'Segment ','Hitpath Offer ID'], right_on=['new Affliate ID', 'Segments','Hitpath Offer ID'],copy = False)
#calculate revenue: estimate eCPM * last 7 days segment size / 1000 
future_deli["project revenue"] = future_deli['avg_eCPM'] * future_deli['Delivered']/1000
future_deli = payout_transform(future_deli)
# future_deli.loc[future_deli['Segment ']=='NO30',"project revenue"] = future_deli['Avg Conversions'] * future_deli['Payout_rate']
# Find the last date of this week, this month, this year and today 
end_of_week = pd.Timestamp(dt.date.today() + relativedelta(weekday=MO))
end_of_month = pd.Timestamp(dt.date.today() + relativedelta(months=1, day=1))
end_of_today = pd.Timestamp(dt.date.today() + relativedelta(days=1))
end_of_year = pd.Timestamp(dt.date.today() + relativedelta(years=1,month=1,day=1))
# Find out the upcoming schedule within current budget periods. For example, we only need all this month's upcoming schedule for a monthly-budgeted offer
offer_future_deli  = future_deli[((future_deli['Cap'] == 'Yearly') & (future_deli['Date'] < end_of_year)) |
                                 ((future_deli['Cap'] == 'Monthly') & (future_deli['Date'] < end_of_month)) |
                                 ((future_deli['Cap'] == 'Weekly') & (future_deli['Date'] < end_of_week)) |
                                 ((future_deli['Cap'] == 'Daily') & (future_deli['Date'] < end_of_today)) |
                                 ((future_deli['Cap'] == 'nan') | (future_deli['Cap'] == 'Test')  | (future_deli['Cap'] == 'Lifetime') | (future_deli['Cap'] == 'PubID Lifetime'))]

offer_future_rev_sum = offer_future_deli.groupby('Hitpath Offer ID').agg({'project revenue': 'sum'})
temp = offer_future_deli.groupby('Hitpath Offer ID').count()['Date'].reset_index(name ='Upcoming No.drops')
offer_future_rev_sum = offer_future_rev_sum.merge(temp, how ='left', on ='Hitpath Offer ID' )

In [ ]:
#################### Merge Used budget and projected budget together ######################
projection = budget_offers1.merge(offer_future_rev_sum, how='left', on='Hitpath Offer ID')
budget_cal = projection.merge(budget_usedRev, how='left', on='Hitpath Offer ID')

# we have 2 type of budgets, one is based on leads, another is based on revenue such as 9623 $1000 daily budget 
# budget calculation is based on budget type 
budget_cal = payout_transform(budget_cal)
budget_cal['Revenue-based Budget'] = budget_cal['Budget'].str.contains(pat = '\$')
budget_cal['Budget'] = budget_cal['Budget'].str.replace(",","")
budget_cal['Budget'] = budget_cal['Budget'].astype('str').str.extract(pat = '([0-9\.]+)',expand = True).astype('float')
budget_cal.loc[budget_cal['Revenue-based Budget'] == True, 'Payout_rate']  =  1

budget_cal['Used_Budget'] = budget_cal['Budget_Used_Revenue'] / budget_cal['Payout_rate']
budget_cal.loc[budget_cal['Payout Type'] == 'variable','Used_Budget' ] = budget_cal['budget_used']
budget_cal['Budget_on_Schedule'] = budget_cal['project revenue'] / budget_cal['Payout_rate']

budget_cal['Used_Budget'] = budget_cal['Used_Budget'].apply(np.ceil)
budget_cal['Budget_on_Schedule'] = budget_cal['Budget_on_Schedule'].apply(np.ceil)


budget_cal['Budget'] = budget_cal['Budget'].astype('str').str.extract(pat = '([0-9\.]+)',expand = True).astype('float')
budget_cal['Project_Remain_Budget'] = budget_cal['Budget'] - budget_cal['Used_Budget'].fillna(0)  -budget_cal['Budget_on_Schedule'].fillna(0)
budget_cal['Budget Used %'] = budget_cal['Used_Budget'].fillna(0)  / budget_cal['Budget'] 
budget_cal['Project Budget Use %'] = (budget_cal['Used_Budget'].fillna(0) + budget_cal['Budget_on_Schedule'].fillna(0)) / budget_cal['Budget'] 

budget_cal_final = budget_cal.loc[:, ["Hitpath Offer ID",'Offer Name','Status','RX Rep','Advertiser','Payout Type','Payout_rate','Upcoming No.drops','Revenue-based Budget',"Budget","Cap",'Budget_Used_Revenue','Used_Budget','Budget_on_Schedule','Project_Remain_Budget','Budget Used %','Project Budget Use %']]

df_mapping = pd.DataFrame({
    'Status': ['Live', 'In Testing', 'Retest']
})
sort_mapping = df_mapping.reset_index().set_index('Status')
budget_cal_final['Operational Status num'] = budget_cal_final['Status'].map(sort_mapping['index'])
budget_cal_final = budget_cal_final.sort_values(['Operational Status num','Advertiser','Project_Remain_Budget'])
budget_cal_final = budget_cal_final.drop('Operational Status num', 1)

offer_future_deli_1 = offer_future_deli.merge(pub_config1, how = 'left', copy =False )
offer_future_deli_1 = offer_future_deli_1[['DP&Pub','Date', 'Dataset', 'Hitpath Offer ID','Avg Conversions','Segment ', 'Delivered',"#Drops","eCPM Metrics","avg_eCPM" ,'project revenue', 'DMA']]

In [ ]:
###############################################################################################################################
#######################  Get Swap Suggestions                          ###################
###############################################################################################################################

swap_suggestions = budget_cal_final.merge(offer_future_deli_1,how = 'left', on = 'Hitpath Offer ID',copy = False)
swap_suggestions = swap_suggestions.merge(budget_used_by_pubid_lifetime, on= ['DP&Pub','Hitpath Offer ID'],how = 'left' )
swap_suggestions.loc[swap_suggestions['Budget_Used_Revenue_by_pubid'].isna() == False,'Budget_Used_Revenue']= swap_suggestions['Budget_Used_Revenue_by_pubid']
swap_suggestions.loc[swap_suggestions['Budget_Used_Revenue_by_pubid'].isna() == False,'Used_Budget']= swap_suggestions['Budget_Used_Revenue_by_pubid']/ swap_suggestions['Payout_rate']

swap_suggestions['Projected_Budget_by_pubid'] = swap_suggestions['project revenue'] / swap_suggestions['Payout_rate']

swap_suggestions.loc[swap_suggestions['Segment ']=='NO30','Projected_Budget_by_pubid'] = swap_suggestions['Avg Conversions'] 
swap_suggestions['Projected_Budget_by_pubid'] = swap_suggestions['Projected_Budget_by_pubid'].round(0)

swap_check = swap_suggestions.copy()

# New Swap Selection Method
swap_suggestions[['Projected_Budget_by_pubid','Used_Budget','Budget_Used_Revenue']] = swap_suggestions[['Projected_Budget_by_pubid','Used_Budget','Budget_Used_Revenue']].fillna(0)
swap_suggestions = swap_suggestions.sort_values(by=['Hitpath Offer ID','Date','Projected_Budget_by_pubid'],ascending=[True,True,False])
swap_suggestions['Projected cumulative budget use'] = ''
for hitpath in swap_suggestions['Hitpath Offer ID'].unique():
    swap_suggestions1 = swap_suggestions[swap_suggestions['Hitpath Offer ID']==hitpath]
    projected_budget = swap_suggestions1['Used_Budget'].unique()[0]
    budget = swap_suggestions1['Budget'].unique()[0]
    for index, row in swap_suggestions1.iterrows():
        if projected_budget + swap_suggestions1.loc[index]['Projected_Budget_by_pubid'] < budget:
            projected_budget = projected_budget + swap_suggestions.loc[index]['Projected_Budget_by_pubid']
            swap_suggestions.loc[index,'Projected cumulative budget use'] = projected_budget

swap_suggestions = swap_suggestions.sort_values(by=['Hitpath Offer ID','Projected cumulative budget use'])
swap_suggestions.reset_index(drop=True, inplace=True)
for hitpath in swap_suggestions['Hitpath Offer ID'].unique():
    swap_suggestions1 = swap_suggestions[swap_suggestions['Hitpath Offer ID']==hitpath]
    for index, row in swap_suggestions1.iterrows():
        if swap_suggestions1.loc[index,'Projected cumulative budget use']=='':
            swap_suggestions.loc[index,'Projected cumulative budget use'] = swap_suggestions.loc[index,'Budget'] + swap_suggestions.loc[index,'Projected_Budget_by_pubid']   

In [ ]:
swap_suggestions

In [ ]:
###############################################################################################################################
#######################  Current Upcoming Schedule                             ###################
###############################################################################################################################

budget_offer_need_to_swap = budget_cal_final.merge(offer_future_deli_1,how = 'left', on = 'Hitpath Offer ID',copy = False)
budget_offer_need_to_swap = budget_offer_need_to_swap.merge(budget_used_by_pubid_lifetime, on= ['DP&Pub','Hitpath Offer ID'],how = 'left' )
budget_offer_need_to_swap.loc[budget_offer_need_to_swap['Budget_Used_Revenue_by_pubid'].isna() == False,'Budget_Used_Revenue']= budget_offer_need_to_swap['Budget_Used_Revenue_by_pubid']
budget_offer_need_to_swap.loc[budget_offer_need_to_swap['Budget_Used_Revenue_by_pubid'].isna() == False,'Used_Budget']= budget_offer_need_to_swap['Budget_Used_Revenue_by_pubid']/ budget_offer_need_to_swap['Payout_rate']

budget_offer_need_to_swap['Projected_Budget_by_pubid'] = budget_offer_need_to_swap['project revenue'] / budget_offer_need_to_swap['Payout_rate']

budget_offer_need_to_swap.loc[budget_offer_need_to_swap['Segment ']=='NO30','Projected_Budget_by_pubid'] = budget_offer_need_to_swap['Avg Conversions'] 
budget_offer_need_to_swap['Projected_Budget_by_pubid'] = budget_offer_need_to_swap['Projected_Budget_by_pubid'].round(0)
budget_offer_need_to_swap['Projected cumulative budget use'] = budget_offer_need_to_swap.groupby(['Hitpath Offer ID',])['Projected_Budget_by_pubid'].cumsum() + budget_offer_need_to_swap['Used_Budget'].fillna(0)
budget_offer_need_to_swap.loc[budget_offer_need_to_swap['Cap'] == 'PubID Lifetime', 'Projected cumulative budget use'] = budget_offer_need_to_swap['Projected_Budget_by_pubid'].fillna(0) + budget_offer_need_to_swap['Used_Budget'].fillna(0)
budget_offer_need_to_swap.loc[budget_offer_need_to_swap['Cap'] == 'PubID Lifetime', 'Budget Used %'] =budget_offer_need_to_swap['Used_Budget'].fillna(0)  / budget_offer_need_to_swap['Budget'] 
budget_offer_need_to_swap.loc[budget_offer_need_to_swap['Cap'] == 'PubID Lifetime', 'Project Budget Use %'] = budget_offer_need_to_swap[ 'Projected cumulative budget use'].fillna(0)  / budget_offer_need_to_swap['Budget'] 

budget_offer_need_to_swap = budget_offer_need_to_swap[['Hitpath Offer ID', 'Offer Name','Status', 'RX Rep','Advertiser','DP&Pub', 'Date', 'DMA' ,'Avg Conversions','Segment ', 'Delivered',"#Drops","eCPM Metrics","avg_eCPM" ,'project revenue','Projected_Budget_by_pubid','Revenue-based Budget',"Budget","Cap",'Used_Budget','Projected cumulative budget use','Budget Used %','Project Budget Use %']]
budget_offer_need_to_swap = budget_offer_need_to_swap[budget_offer_need_to_swap['DP&Pub'].isna() == False]
budget_offer_need_to_swap['Avg Conversions'] = budget_offer_need_to_swap['Avg Conversions'].round(1)

budget_offer_need_to_swap['project revenue'] = budget_offer_need_to_swap['project revenue'].apply(lambda x: "${:.1f}".format(x) if  pd.isna(x) == False else x)


budget_offer_need_to_swap = budget_offer_need_to_swap.rename({'Delivered':'Estimated Delivered', 'project revenue':'Projected Revenue'},axis = 1)
budget_offer_need_to_swap['#Drops'] = budget_offer_need_to_swap['#Drops'].fillna(0)
budget_offer_need_to_swap_within_budget_period  = budget_offer_need_to_swap[((budget_offer_need_to_swap['Cap'] == 'Yearly') & (budget_offer_need_to_swap['Date'] < end_of_year)) | 
                                ((budget_offer_need_to_swap['Cap'] == 'Monthly') & (budget_offer_need_to_swap['Date'] < end_of_month)) | 
                               ((budget_offer_need_to_swap['Cap'] == 'Weekly') & (budget_offer_need_to_swap['Date'] < end_of_week)) |
                               ((budget_offer_need_to_swap['Cap'] == 'Daily') & (budget_offer_need_to_swap['Date'] < end_of_today)) |
                               ((budget_offer_need_to_swap['Cap'] == 'nan') | (budget_offer_need_to_swap['Cap'] == 'Test')  | (budget_offer_need_to_swap['Cap'] == 'Lifetime') | (budget_offer_need_to_swap['Cap'] == 'PubID Lifetime'))]


# for the offers that is almost run out of budget 
budget_offer_need_to_swap_1 = budget_offer_need_to_swap_within_budget_period.loc[budget_offer_need_to_swap_within_budget_period['Project Budget Use %']>= 0.95]
budget_cal_final['Payout_rate'] = budget_cal_final['Payout_rate'].apply(lambda x: "${:.1f}".format(x) if pd.isna(x) == False else x)
budget_cal_final['Budget_Used_Revenue'] = budget_cal_final['Budget_Used_Revenue'].apply(lambda x: "${:.1f}".format(x) if  pd.isna(x) == False else x)
budget_cal_final.loc[budget_cal_final['Revenue-based Budget'] == True, 'Used_Budget']  =  budget_cal_final['Used_Budget'].apply(lambda x: "${:.1f}".format(x) if  pd.isna(x) == False else x)
budget_cal_final.loc[budget_cal_final['Revenue-based Budget'] == True, 'Budget']  =  budget_cal_final['Budget'].apply(lambda x: "${:.1f}".format(x) if  pd.isna(x) == False else x)
budget_offer_need_to_swap_1['Date']= budget_offer_need_to_swap_1['Date'].dt.strftime('%Y-%m-%d')
budget_offer_need_to_swap_within_budget_period['Date']= budget_offer_need_to_swap_within_budget_period['Date'].dt.strftime('%Y-%m-%d')

# create Legend to explain color code 
legend = {'Color Code':['Blue','Green','Yellow','Orange','Red','Red Text','Blue Text'],'Meaning':['Please follow the Budget Allocation Sheet','Free to Send','Budget Remaining (Allocation Request Needed)','Budget Allocated','No Budget Remaining','Not enough budget for the drop','The offer has no more than 1 drops in the DP&Pub']}
legend_df = pd.DataFrame.from_dict(legend)
#budget_cal_final.to_csv('budget_report.csv', index = False)

In [ ]:
###############################################################################################################################
####################### All upcoming drops                                                                  ###################
###############################################################################################################################
temp = future_deli.groupby('Hitpath Offer ID')['Date'].min().reset_index(name = 'Budget Period Start Date' )
future_deli1 = future_deli.merge(temp, on ='Hitpath Offer ID', how = 'left')
future_deli1.loc[future_deli['Cap'] == 'Monthly','Budget Period Start Date'] = future_deli1['Date'].dt.to_period('M').dt.to_timestamp()
future_deli1.loc[future_deli['Cap'] == 'Weekly','Budget Period Start Date'] = future_deli1['Date'].dt.to_period('W').dt.to_timestamp()
future_deli1.loc[future_deli['Cap'] == 'Daily','Budget Period Start Date'] = future_deli1['Date'].dt.to_period('D').dt.to_timestamp()
future_deli1.loc[future_deli['Cap'] == 'Yearly','Budget Period Start Date'] = future_deli1['Date'].dt.to_period('Y').dt.to_timestamp()
# future_deli1.loc[((future_deli1['Cap'] == 'nan') | (future_deli1['Cap'] == 'Test')  | (future_deli1['Cap'] == 'Lifetime') | (future_deli1['Cap'] == 'PubID Lifetime'))& (future_deli1['Budget Start Date'].isna() == False),'Budget Period Start Date'] = future_deli1['Budget Start Date']

future_rev_sum = future_deli1.groupby(['Hitpath Offer ID','Budget Period Start Date']).agg({'project revenue': 'sum'}) 
temp = future_deli1.groupby(['Hitpath Offer ID','Budget Period Start Date']).count()['Date'].reset_index(name ='Upcoming No.drops')
future_rev_sum = future_rev_sum.merge(temp, how ='left', on =['Hitpath Offer ID','Budget Period Start Date'] )

####merge back to the budget offer sheet
Projection = budget_offers1.merge(future_rev_sum, how='left', on='Hitpath Offer ID')

budget_cal_all_upcoming_drops = Projection.merge(budget_usedRev, how='left', on='Hitpath Offer ID')

budget_cal_all_upcoming_drops = payout_transform(budget_cal_all_upcoming_drops)
budget_cal_all_upcoming_drops['Budget_Used_Revenue_for_Budget_Period'] = 0 
budget_cal_all_upcoming_drops.loc[(budget_cal_all_upcoming_drops['Cap'] == 'Monthly') & (budget_cal_all_upcoming_drops['Budget Period Start Date'] == start_of_month),'Budget_Used_Revenue_for_Budget_Period'] = budget_cal_all_upcoming_drops['Budget_Used_Revenue']
budget_cal_all_upcoming_drops.loc[(budget_cal_all_upcoming_drops['Cap'] == 'Weekly') & (budget_cal_all_upcoming_drops['Budget Period Start Date'] == start_of_week),'Budget_Used_Revenue_for_Budget_Period'] = budget_cal_all_upcoming_drops['Budget_Used_Revenue']
budget_cal_all_upcoming_drops.loc[(budget_cal_all_upcoming_drops['Cap'] == 'Daily') & (budget_cal_all_upcoming_drops['Budget Period Start Date'] == start_of_today),'Budget_Used_Revenue_for_Budget_Period'] = budget_cal_all_upcoming_drops['Budget_Used_Revenue']
budget_cal_all_upcoming_drops.loc[(budget_cal_all_upcoming_drops['Cap'] == 'nan') | (budget_cal_all_upcoming_drops['Cap'] == 'Test')  | (budget_cal_all_upcoming_drops['Cap'] == 'Lifetime'),'Budget_Used_Revenue_for_Budget_Period'] = budget_cal_all_upcoming_drops['Budget_Used_Revenue']


budget_cal_all_upcoming_drops['Revenue-based Budget'] = budget_cal_all_upcoming_drops['Budget'].str.contains(pat = '\$')
budget_cal_all_upcoming_drops['Budget'] = budget_cal_all_upcoming_drops['Budget'].str.replace(",","")
budget_cal_all_upcoming_drops['Budget'] = budget_cal_all_upcoming_drops['Budget'].astype('str').str.extract(pat = '([0-9\.]+)',expand = True).astype('float')
budget_cal_all_upcoming_drops.loc[budget_cal_all_upcoming_drops['Revenue-based Budget'] == True, 'Payout_rate']  =  1


budget_cal_all_upcoming_drops['Used_Budget'] = budget_cal_all_upcoming_drops['Budget_Used_Revenue_for_Budget_Period'] / budget_cal_all_upcoming_drops['Payout_rate']
budget_cal_all_upcoming_drops.loc[budget_cal_all_upcoming_drops['Payout Type'] == 'variable','Used_Budget' ] = budget_cal_all_upcoming_drops['budget_used']
budget_cal_all_upcoming_drops['Budget_on_Schedule'] = budget_cal_all_upcoming_drops['project revenue'] / budget_cal_all_upcoming_drops['Payout_rate']

budget_cal_all_upcoming_drops['Used_Budget'] = budget_cal_all_upcoming_drops['Used_Budget'].apply(np.ceil)
budget_cal_all_upcoming_drops['Budget_on_Schedule'] = budget_cal_all_upcoming_drops['Budget_on_Schedule'].apply(np.ceil)

budget_cal_all_upcoming_drops['Budget'] = budget_cal_all_upcoming_drops['Budget'].astype('str').str.extract(pat = '([0-9\.]+)',expand = True).astype('float')
budget_cal_all_upcoming_drops['Project_Remain_Budget'] = budget_cal_all_upcoming_drops['Budget'] - budget_cal_all_upcoming_drops['Used_Budget'].fillna(0)  -budget_cal_all_upcoming_drops['Budget_on_Schedule'].fillna(0)
budget_cal_all_upcoming_drops['Budget Used %'] = budget_cal_all_upcoming_drops['Used_Budget'].fillna(0)  / budget_cal_all_upcoming_drops['Budget'] 
budget_cal_all_upcoming_drops['Project Budget Use %'] = (budget_cal_all_upcoming_drops['Used_Budget'].fillna(0) + budget_cal_all_upcoming_drops['Budget_on_Schedule'].fillna(0)) / budget_cal_all_upcoming_drops['Budget'] 

budget_cal_all_upcoming_drops_final = budget_cal_all_upcoming_drops.loc[:, ["Hitpath Offer ID",'Offer Name','Status','RX Rep','Advertiser','Payout Type','Payout_rate','Upcoming No.drops','Revenue-based Budget',"Budget","Cap",'Budget_Used_Revenue_for_Budget_Period','Budget Period Start Date','Used_Budget','Budget_on_Schedule','Project_Remain_Budget','Budget Used %','Project Budget Use %']]

future_deli_2 = future_deli1.merge(pub_config1, how = 'left', copy =False )
future_deli_2 = future_deli_2[['DP&Pub','Date', 'Dataset', 'Hitpath Offer ID','Avg Conversions','Segment ', 'Delivered',"#Drops","eCPM Metrics","avg_eCPM" ,'project revenue','DMA','Budget Period Start Date']]

all_upcoming_drops = future_deli_2.merge(budget_cal_all_upcoming_drops_final,how = 'left', on = 'Hitpath Offer ID',copy = False)
all_upcoming_drops = all_upcoming_drops[(all_upcoming_drops['Budget Period Start Date_x'] == all_upcoming_drops['Budget Period Start Date_y']) | (all_upcoming_drops['Budget Period Start Date_x'].isna() & all_upcoming_drops['Budget Period Start Date_y'].isna()) ] 
all_upcoming_drops['Budget_Used_Revenue_for_Budget_Period']  = all_upcoming_drops['Budget_Used_Revenue_for_Budget_Period'].fillna(0)
all_upcoming_drops['Used_Budget']  = all_upcoming_drops['Used_Budget'].fillna(0)


all_upcoming_drops = all_upcoming_drops.merge(budget_used_by_pubid_lifetime, on= ['DP&Pub','Hitpath Offer ID'],how = 'left' )
all_upcoming_drops.loc[all_upcoming_drops['Budget_Used_Revenue_by_pubid'].isna() == False,'Budget_Used_Revenue']= all_upcoming_drops['Budget_Used_Revenue_by_pubid']
all_upcoming_drops.loc[all_upcoming_drops['Budget_Used_Revenue_by_pubid'].isna() == False,'Used_Budget']= all_upcoming_drops['Budget_Used_Revenue_by_pubid']/ all_upcoming_drops['Payout_rate']

all_upcoming_drops['Projected_Budget_by_pubid'] = all_upcoming_drops['project revenue'] / all_upcoming_drops['Payout_rate']
# all_upcoming_drops.loc[all_upcoming_drops['real seg']=='NO30','Projected_Budget_by_pubid'] = all_upcoming_drops['Avg Conversions']
all_upcoming_drops['Projected_Budget_by_pubid'] = all_upcoming_drops['Projected_Budget_by_pubid'].round(0)
all_upcoming_drops = all_upcoming_drops.rename(columns= {'Budget Period Start Date_x':'Budget Period Start Date'})
all_upcoming_drops = all_upcoming_drops.sort_values(['Date','Projected_Budget_by_pubid'])
all_upcoming_drops['Projected cumulative budget use'] = all_upcoming_drops.groupby(['Hitpath Offer ID','Budget Period Start Date'])['Projected_Budget_by_pubid'].cumsum() + all_upcoming_drops['Used_Budget'].fillna(0)

all_upcoming_drops = all_upcoming_drops[['Hitpath Offer ID', 'Offer Name','Status', 'RX Rep','Advertiser','DP&Pub', 'Date', 'DMA' ,'Avg Conversions','Segment ', 'Delivered',"#Drops","eCPM Metrics","avg_eCPM" ,'project revenue','Projected_Budget_by_pubid','Revenue-based Budget',"Budget","Cap",'Budget Period Start Date','Used_Budget','Projected cumulative budget use','Budget Used %','Project Budget Use %']]
all_upcoming_drops = all_upcoming_drops[all_upcoming_drops['DP&Pub'].isna() == False]
all_upcoming_drops['Avg Conversions'] = all_upcoming_drops['Avg Conversions'].round(1)


all_upcoming_drops['project revenue'] = all_upcoming_drops['project revenue'].apply(lambda x: "${:.1f}".format(x) if  pd.isna(x) == False else x)
all_upcoming_drops.loc[all_upcoming_drops['Cap'] == 'PubID Lifetime', 'Projected cumulative budget use'] = all_upcoming_drops['Projected_Budget_by_pubid'].fillna(0) + all_upcoming_drops['Used_Budget'].fillna(0)
all_upcoming_drops.loc[all_upcoming_drops['Cap'] == 'PubID Lifetime', 'Budget Used %'] =all_upcoming_drops['Used_Budget'].fillna(0)  / all_upcoming_drops['Budget'] 
all_upcoming_drops.loc[all_upcoming_drops['Cap'] == 'PubID Lifetime', 'Project Budget Use %'] = all_upcoming_drops[ 'Projected cumulative budget use'].fillna(0) / all_upcoming_drops['Budget'] 



all_upcoming_drops = all_upcoming_drops.rename({'Delivered':'Estimated Delivered', 'project revenue':'Projected Revenue'},axis = 1)
all_upcoming_drops['#Drops'] = all_upcoming_drops['#Drops'].fillna(0)
df_mapping = pd.DataFrame({
    'Status': ['Live', 'In Testing', 'Retest']
})
sort_mapping = df_mapping.reset_index().set_index('Status')
all_upcoming_drops['Operational Status num'] = all_upcoming_drops['Status'].map(sort_mapping['index'])
all_upcoming_drops = all_upcoming_drops.sort_values(['Operational Status num','Advertiser','Hitpath Offer ID'])
all_upcoming_drops = all_upcoming_drops.drop('Operational Status num', 1)
all_upcoming_drops.loc[all_upcoming_drops['Cap'].isna(), ['Budget Period Start Date','Used_Budget','Budget Used %']] = np.nan 

all_upcoming_drops['Budget Period Start Date']= all_upcoming_drops['Budget Period Start Date'].dt.strftime('%Y-%m-%d')
all_upcoming_drops['Date']= all_upcoming_drops['Date'].dt.strftime('%Y-%m-%d')

In [ ]:
# ###############################################################################################################################
# #######################               Budgeted Allocation for Offer ID                                      ###################
# ###############################################################################################################################
# budgeted_offer_list = offers[offers['Operational Status'] == 'Budgeted Offer - Budget Allocated']['Hitpath Offer ID'].tolist()
# def available_list(hitpath): 
#     paused_list = offers[offers['Hitpath Offer ID']==hitpath]['Paused PubIDs'].values[0]
#     live_list = offers[offers['Hitpath Offer ID']==hitpath]['Live PubIDs'].values[0]
    
#     available_datasets = list(set(emit['new Affiliate ID'].unique().tolist()))
    
#     if live_list is not np.nan: 
#         live_datasets =  offers[offers['Hitpath Offer ID']==hitpath]['Live PubIDs'].str.split('\n| |\t|,').values[0]
#         available_datasets = (set(live_datasets) & set(available_datasets))
#         if '460398' in available_datasets:
#             available_datasets.remove('460398')
#             available_datasets += ['SC.RF','SC.FHA']
#         if '461128' in available_datasets:
#             available_datasets.remove('461128')
#             available_datasets += ['LPG.RF','LPG.FHA'] 
    
#     if paused_list is not np.nan: 
#         paused_list1 = offers[offers['Hitpath Offer ID']==hitpath]['Paused PubIDs'].str.split('\n| |\t|,').values[0]
#         if '460398' in paused_list1:
#             paused_list1.remove('460398')
#             paused_list1 += ['SC.RF','SC.FHA']
#         if '461128' in paused_list1:
#             paused_list1.remove('461128')
#             paused_list1 += ['LPG.RF','LPG.FHA'] 
#         available_datasets = list(set(available_datasets) - set(paused_list1))
#     budgeted_offer_df = pd.DataFrame(available_datasets, columns = ['new Affliate ID'])
#     budgeted_offer_df['Hitpath Offer ID'] = hitpath
#     budgeted_offer_df = budgeted_offer_df.merge(avgDelivered, how = 'left', copy = False)
#     budgeted_offer_df = budgeted_offer_df.merge(account_eCPM, how = 'left', copy = False)
#     budgeted_offer_df["project revenue"] = budgeted_offer_df['avg_eCPM'] * budgeted_offer_df['Delivered']/1000
#     budgeted_offer_df = budgeted_offer_df[budgeted_offer_df['avg_eCPM'].isna() == False]
    
#     budgeted_offer_df['avg_eCPM'] = budgeted_offer_df['avg_eCPM'].apply(lambda x: "${:.1f}".format(x) if  pd.isna(x) == False else x)
#     budgeted_offer_df = budgeted_offer_df.merge(budget_cal[['Hitpath Offer ID','Revenue-based Budget','Payout_rate']],how='left')
#     budgeted_offer_df['project Budget'] = budgeted_offer_df['project revenue'] / budgeted_offer_df['Payout_rate']
#     budgeted_offer_df.loc[budgeted_offer_df['Revenue-based Budget'] == True, 'project Budget']  =  budgeted_offer_df['project Budget'].apply(lambda x: "${:.1f}".format(x) if  pd.isna(x) == False else x)
#     budgeted_offer_df = budgeted_offer_df.drop(['Payout_rate'],axis = 1)
#     budgeted_offer_df['project revenue'] = budgeted_offer_df['project revenue'].apply(lambda x: "${:.1f}".format(x) if  pd.isna(x) == False else x)
#     return budgeted_offer_df

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders

In [ ]:
os.chdir('/Users/nathan/Documents/SMS DMA Python/SMS DataSource/output')
with pd.ExcelWriter('swap_check.xlsx') as writer:
    swap_check.to_excel(writer, sheet_name='swap_check')

def readfile(path):
    file = open(path)
    str = file.read()
    return str

def send_email2(filename,toaddr):

    fromaddr = 'nathan@rxmg.com'

    # creates SMTP session 
    s = smtplib.SMTP('smtp.gmail.com', 587) 

    # start TLS for security 
    s.starttls() 

    # Authentication 
    s.login(fromaddr, 'sgtyjqzppzoltuau') 

    # instance of MIMEMultipart 
    msg = MIMEMultipart() 

    # storing the senders email address   
    msg['From'] = 'nathan@rxmg.com'

    # storing the receivers email address  
    msg['To'] = toaddr

    # storing the subject  
    msg['Subject'] = 'Budget Report Swap Check'

    # string to store the body of the mail 
    body = ''

    # attach the body with the msg instance 
    msg.attach(MIMEText(body, 'plain'))

    # open the file to be sent  
    attachment = open(filename, "rb") 

    # instance of MIMEBase and named as p 
    p = MIMEBase('application', 'octet-stream') 

    # To change the payload into encoded form 
    p.set_payload((attachment).read()) 

    # encode into base64 
    encoders.encode_base64(p) 

    p.add_header('Content-Disposition', "attachment; filename= %s" % filename) 

    # attach the instance 'p' to instance 'msg' 
    msg.attach(p) 

    # Converts the Multipart msg into a string 
    text = msg.as_string() 

    # sending the mail 
    s.sendmail(fromaddr, toaddr, text) 

    # terminating the session 
    s.quit()

address_list = ['nathan@rxmg.com']
for address in address_list:
    send_email2('swap_check.xlsx',address)

In [ ]:
###############################################################################################################################
####################### Please change to your local directory to store the files in the following part!     ###################
###############################################################################################################################

os.chdir('/Users/nathan/Documents/SMS DMA Python/SMS DataSource/Budget_Report')
date_time = today.strftime('%m-%d-%Y')
filepath = f'Budget_Report-{date_time}.xlsx'
writer_budgetReport = pd.ExcelWriter(filepath, engine = 'xlsxwriter')
workbook_budgetReport = writer_budgetReport.book
am_list = budget_cal_final['RX Rep'].unique().tolist()

In [ ]:
###############################################################################################################################
#######################               Worksheet Format and write into excel                                 ###################
###############################################################################################################################


def worksheet_format(workbook,worksheet):
    format_red = workbook.add_format({'bg_color':   '#F5B7B1','num_format': '0.00%'})
    format_orange = workbook.add_format({'bg_color':   '#F5CBA7','num_format': '0.00%'})
    format_yellow = workbook.add_format({'bg_color':   '#FCF3CF','num_format': '0.00%'})
    format_green = workbook.add_format({'bg_color':   '#D4EFDF','num_format': '0.00%'})
    bg_blue =  workbook.add_format({'bg_color':   '#D6EAF8','num_format': '0.00%'})
    formatdate = {'num_format':'yyyy-mm-dd'}
    front_red = workbook.add_format({'font_color': '#E74C3C'})
    format1 = workbook.add_format({'font_color': '#0617d1'})
    bg_red =  workbook.add_format({'bg_color':   '#F5B7B1'})
    bg_orange =  workbook.add_format({'bg_color':   '#F5CBA7'})
    bg_yellow =  workbook.add_format({'bg_color':   '#FCF3CF'})
    bg_green =  workbook.add_format({'bg_color':   '#D4EFDF'})
    curr_format=workbook.add_format({'num_format':'$#,##0.00'})



    worksheet.conditional_format('V2:V1048576', {'type':     'formula',
                                       'criteria': '=$V2 >= $S2',
                                       'format':   front_red})   
    worksheet.conditional_format('Q1:S1048576', {'type':     'formula',
                                       'criteria':  '=$R1 = TRUE',
                                       'format':   curr_format})  
    worksheet.conditional_format('W1:X1048576', {'type':     'formula',
                                       'criteria':  '=AND($W1<> "Budget Used %",$W1>=0.95)',
                                       'format':   format_red})
    worksheet.conditional_format('W1:X1048576', {'type':     'formula',
                                       'criteria': '=AND($X1<> "Project Budget Use %",$X1>=0.95)',
                                       'format':   format_orange})
    worksheet.conditional_format('W1:X1048576', {'type':     'formula',
                                       'criteria': '=AND($X1>=0.7,$X1<0.95)',
                                       'format':   format_yellow})
    worksheet.conditional_format('W1:X1048576', {'type':     'formula',
                                       'criteria': '=AND($X1 <> "",AND($X1>=0,$X1<0.70))',
                                       'format':   format_green})
    worksheet.conditional_format('Z4', {'type':     'formula',
                                       'criteria': '=$Z4 = "Blue"',
                                       'format':   bg_blue} )   
    worksheet.conditional_format('Z5', {'type':     'formula',
                                       'criteria': '=$Z5 = "Green"',
                                       'format':   bg_green} )                     
    worksheet.conditional_format('Z6', {'type':     'formula',
                                       'criteria': '=$Z6 = "Yellow"',
                                       'format':   bg_yellow} )  
    worksheet.conditional_format('Z7', {'type':     'formula',
                                       'criteria': '=$Z7 = "Orange"',
                                       'format':   bg_orange} )  
    worksheet.conditional_format('Z8', {'type':     'formula',
                                       'criteria': '=$Z8 = "Red"',
                                       'format':   bg_red} )
    worksheet.conditional_format('Z9', {'type':     'formula',
                                       'criteria': '=$Z9 = "Red Text"',
                                       'format':   front_red} )  
    worksheet.conditional_format('Z10', {'type':     'formula',
                                       'criteria': '=$Z10 = "Blue Text"',
                                       'format':   format1} )          
    worksheet.conditional_format('M1:M1048576', {'type':     'formula',
                                       'criteria':  '=AND($M1>=0,$M1<=1)',
                                       'format':   format1})
    worksheet.set_column(1, 23, 11)
   


                               
def worksheet_format_am(workbook,worksheet):
    format_red = workbook.add_format({'bg_color':   '#F5B7B1','num_format': '0.00%'})
    format_orange = workbook.add_format({'bg_color':   '#F5CBA7','num_format': '0.00%'})
    format_yellow = workbook.add_format({'bg_color':   '#FCF3CF','num_format': '0.00%'})
    format_green = workbook.add_format({'bg_color':   '#D4EFDF','num_format': '0.00%'})
    bg_blue =  workbook.add_format({'bg_color':   '#D6EAF8','num_format': '0.00%'})
    bg_red =  workbook.add_format({'bg_color':   '#F5B7B1'})
    format1 = workbook.add_format({'font_color': '#0617d1'})
    bg_orange =  workbook.add_format({'bg_color':   '#F5CBA7'})
    bg_yellow =  workbook.add_format({'bg_color':   '#FCF3CF'})
    bg_green =  workbook.add_format({'bg_color':   '#D4EFDF'})
    front_red = workbook.add_format({'font_color': '#E74C3C'})


    worksheet.conditional_format('Q1:R1048576', {'type':     'formula',
                                       'criteria': '=$C1 = "Budgeted Offer - Budget Allocated"',
                                       'format':   bg_blue})
    worksheet.conditional_format('Q1:R1048576', {'type':     'formula',
                                       'criteria':  '=AND($Q1<> "Budget Used %",$Q1>=0.95)',
                                       'format':   format_red})
    worksheet.conditional_format('Q1:R1048576', {'type':     'formula',
                                       'criteria': '=AND($R1<> "Project Budget Use %",$R1>=0.95)',
                                       'format':   format_orange})
    worksheet.conditional_format('Q1:R1048576', {'type':     'formula',
                                       'criteria': '=AND($R1>=0.7,$R1<0.95)',
                                       'format':   format_yellow})
    worksheet.conditional_format('Q1:R1048576', {'type':     'formula',
                                       'criteria': '=AND($R1 <> "",AND($R1>=0,$R1<0.70))',
                                       'format':   format_green})

    worksheet.conditional_format('U4', {'type':     'formula',
                                       'criteria': '=$U4 = "Blue"',
                                       'format':   bg_blue} )   
    
    worksheet.conditional_format('U5', {'type':     'formula',
                                       'criteria': '=$U5 = "Green"',
                                       'format':   bg_green} )                     
    worksheet.conditional_format('U6', {'type':     'formula',
                                       'criteria': '=$U6 = "Yellow"',
                                       'format':   bg_yellow} )  
    worksheet.conditional_format('U7', {'type':     'formula',
                                       'criteria': '=$U7 = "Orange"',
                                       'format':   bg_orange} )  
    worksheet.conditional_format('U8', {'type':     'formula',
                                       'criteria': '=$U8 = "Red"',
                                       'format':   bg_red} )                                    
    worksheet.conditional_format('U9', {'type':     'formula',
                                       'criteria': '=$U9 = "Red Text"',
                                       'format':   front_red} )
    worksheet.conditional_format('U10', {'type':     'formula',
                                       'criteria': '=$U10 = "Blue Text"',
                                       'format':   format1} )  
    worksheet.set_column(1, 18, 11)
    
def worksheet_format_all_upcoming_schedule(workbook,worksheet):

    format_red = workbook.add_format({'bg_color':   '#F5B7B1','num_format': '0.00%'})
    format_orange = workbook.add_format({'bg_color':   '#F5CBA7','num_format': '0.00%'})
    format_yellow = workbook.add_format({'bg_color':   '#FCF3CF','num_format': '0.00%'})
    format_green = workbook.add_format({'bg_color':   '#D4EFDF','num_format': '0.00%'})
    bg_blue =  workbook.add_format({'bg_color':   '#D6EAF8','num_format': '0.00%'})
    format1 = workbook.add_format({'font_color': '#0617d1'})
    bg_red =  workbook.add_format({'bg_color':   '#F5B7B1'})
    bg_orange =  workbook.add_format({'bg_color':   '#F5CBA7'})
    bg_yellow =  workbook.add_format({'bg_color':   '#FCF3CF'})
    bg_green =  workbook.add_format({'bg_color':   '#D4EFDF'})
    curr_format=workbook.add_format({'num_format':'$#,##0.00'})
    front_red = workbook.add_format({'font_color': '#E74C3C'})




    worksheet.conditional_format('W2:W1048576', {'type':     'formula',
                                       'criteria': '=$W2 >= $S2',
                                       'format':   front_red})   
    worksheet.conditional_format('X1:Y1048576', {'type':     'formula',
                                       'criteria': '=$C1 = "Budgeted Offer - Budget Allocated"',
                                       'format':   bg_blue})
    worksheet.conditional_format('Q1:S1048576', {'type':     'formula',
                                       'criteria':  '=$R1 = TRUE' ,
                                       'format':   curr_format})  
    
    worksheet.conditional_format('X1:Y1048576', {'type':     'formula',
                                       'criteria':  '=AND($X1<> "Budget Used %",$X1>=0.95)',
                                       'format':   format_red})
    worksheet.conditional_format('X1:Y1048576', {'type':     'formula',
                                       'criteria': '=AND($Y1<> "Project Budget Use %",$Y1>=0.95)',
                                       'format':   format_orange})
    worksheet.conditional_format('X1:Y1048576', {'type':     'formula',
                                       'criteria': '=AND($Y1>=0.7,$Y1<0.95)',
                                       'format':   format_yellow})
    worksheet.conditional_format('X1:Y1048576', {'type':     'formula',
                                       'criteria': '=AND($Y1 <> "",AND($Y1>=0,$Y1<0.70))',
                                       'format':   format_green})
    worksheet.conditional_format('Z4', {'type':     'formula',
                                       'criteria': '=$Z4 = "Blue"',
                                       'format':   bg_blue} )   
    worksheet.conditional_format('Z5', {'type':     'formula',
                                       'criteria': '=$Z5 = "Green"',
                                       'format':   bg_green} )                     
    worksheet.conditional_format('Z6', {'type':     'formula',
                                       'criteria': '=$Z6 = "Yellow"',
                                       'format':   bg_yellow} )  
    worksheet.conditional_format('Z7', {'type':     'formula',
                                       'criteria': '=$Z7 = "Orange"',
                                       'format':   bg_orange} )  
    worksheet.conditional_format('Z8', {'type':     'formula',
                                       'criteria': '=$Z8 = "Red"',
                                       'format':   bg_red} )  
    worksheet.conditional_format('Z9', {'type':     'formula',
                                       'criteria': '=$Z9 = "Red Text"',
                                       'format':   front_red} )  
    worksheet.conditional_format('Z10', {'type':     'formula',
                                       'criteria': '=$Z10 = "Blue Text"',
                                       'format':   format1} )  
    worksheet.conditional_format('M1:M1048576', {'type':     'formula',
                                       'criteria':  '=AND($M1>=0,$M1<=1)',
                                       'format':   format1})
    worksheet.set_column(1, 24, 11)
    
header_format = workbook_budgetReport.add_format({'bold': True,'text_wrap': True,'valign': 'top','align':'center','border': 1})                               

offer_with_budget_cap = budget_cal_final[budget_cal_final['Budget'].isna() == False]
offer_without_budget_cap =  budget_cal_final[budget_cal_final['Budget'].isna() == True]
worksheet = workbook_budgetReport.add_worksheet('Budget Offer with Cap')
worksheet_format_am(workbook_budgetReport, worksheet)
writer_budgetReport.sheets['Budget Offer with Cap'] = worksheet


offer_with_budget_cap.to_excel(writer_budgetReport,sheet_name='Budget Offer with Cap',startrow=0 , startcol=0, index = False)
legend_df.to_excel(writer_budgetReport,sheet_name='Budget Offer with Cap',startrow=2 , startcol=20, index = False)
# Write the column headers with the defined format.
for col_num, value in enumerate(offer_with_budget_cap.columns.values):
    worksheet.write(0, col_num, value, header_format)

worksheet = workbook_budgetReport.add_worksheet('Budget Offer without Cap')
worksheet_format_am(workbook_budgetReport, worksheet)
writer_budgetReport.sheets['Budget Offer without Cap'] = worksheet
offer_without_budget_cap.to_excel(writer_budgetReport,sheet_name='Budget Offer without Cap',startrow=0 , startcol=0,  index = False)
legend_df.to_excel(writer_budgetReport,sheet_name='Budget Offer without Cap',startrow=2 , startcol=20, index = False)
# Write the column headers with the defined format.
for col_num, value in enumerate(offer_without_budget_cap.columns.values):
    worksheet.write(0, col_num, value, header_format)



worksheet = workbook_budgetReport.add_worksheet('Budget Offer Need to Swap')

writer_budgetReport.sheets['Budget Offer Need to Swap'] = worksheet
worksheet_format(workbook_budgetReport, worksheet)
budget_offer_need_to_swap_1.to_excel(writer_budgetReport,sheet_name='Budget Offer Need to Swap',startrow=0 , startcol=0,  index = False)
legend_df.to_excel(writer_budgetReport,sheet_name='Budget Offer Need to Swap',startrow=2 , startcol=25, index = False)
# Write the column headers with the defined format.
for col_num, value in enumerate(budget_offer_need_to_swap_1.columns.values):
    worksheet.write(0, col_num, value, header_format)

worksheet = workbook_budgetReport.add_worksheet("Today's Budgeted Drop Schedule")
worksheet_format(workbook_budgetReport, worksheet)
writer_budgetReport.sheets['Current Upcoming Schedule'] = worksheet
budget_offer_need_to_swap_within_budget_period.to_excel(writer_budgetReport,sheet_name='Current Upcoming Schedule',startrow=0 , startcol=0,  index = False)
legend_df.to_excel(writer_budgetReport,sheet_name='Current Upcoming Schedule',startrow=2 , startcol=25, index = False)
# Write the column headers with the defined format.
for col_num, value in enumerate(budget_offer_need_to_swap_within_budget_period.columns.values):
    worksheet.write(0, col_num, value, header_format)


worksheet = workbook_budgetReport.add_worksheet('Upcoming Budgeted Drop Schedule')
worksheet_format_all_upcoming_schedule(workbook_budgetReport, worksheet)
writer_budgetReport.sheets['All Upcoming Schedule'] = worksheet
all_upcoming_drops.to_excel(writer_budgetReport,sheet_name='All Upcoming Schedule',startrow=0 , startcol=0,  index = False)
legend_df.to_excel(writer_budgetReport,sheet_name='All Upcoming Schedule',startrow=2 , startcol=25, index = False)
    # Write the column headers with the defined format.
for col_num, value in enumerate(all_upcoming_drops.columns.values):
    worksheet.write(0, col_num, value, header_format)

In [ ]:
###############################################################################################################################
#######################               specific bugdet sheet for each AM                                     ###################
###############################################################################################################################
for i in am_list:
    am_df = budget_cal_final[budget_cal_final['RX Rep'] == i]
    worksheet = workbook_budgetReport.add_worksheet(f'{i}')
    worksheet_format_am(workbook_budgetReport, worksheet)
    writer_budgetReport.sheets[f'{i}'] = worksheet
    
    am_df.to_excel(writer_budgetReport,sheet_name=f'{i}',startrow=0 , startcol=0, index = False) 
    legend_df.to_excel(writer_budgetReport,sheet_name=f'{i}',startrow=2 , startcol=20, index = False)
        # Write the column headers with the defined format.
    for col_num, value in enumerate(am_df.columns.values):
        worksheet.write(0, col_num, value, header_format)

# for i in budgeted_offer_list:
#     budgeted_offer_df = available_list(i)
#     worksheet = workbook_budgetReport.add_worksheet(f'Budgeted Allocation for {i}')
#     writer_budgetReport.sheets[f'Budgeted Allocation for {i}'] = worksheet
#     budgeted_offer_df.to_excel(writer_budgetReport,sheet_name=f'Budgeted Allocation for {i}',startrow=0 , startcol=0, index = False) 


worksheet = workbook_budgetReport.add_worksheet('Active Pubid')
writer_budgetReport.sheets['Active Pubid'] = worksheet
active_pubid.to_excel(writer_budgetReport,sheet_name='Active Pubid',startrow=0 , startcol=0,  index = False)

writer_budgetReport.save()

In [ ]:
# create content for email body: 
swap_suggestions1 = swap_suggestions.copy()
all_upcoming_drops1 = all_upcoming_drops.merge(sms_offer[['Hitpath Offer ID','Scheduling Name']], how= 'left')
all_upcoming_drops1 = all_upcoming_drops1.drop(columns=['Projected cumulative budget use','Budget'])
swap_suggestions1['Date'] = swap_suggestions1['Date'].astype(str)
all_upcoming_drops1 = all_upcoming_drops1.merge(swap_suggestions1,how='right',on=['Hitpath Offer ID','DP&Pub', 'Date','RX Rep','Cap','DMA','Revenue-based Budget'])
out_of_budget = all_upcoming_drops1.loc[all_upcoming_drops1['Projected cumulative budget use']>= all_upcoming_drops1['Budget']]
out_of_budget_list = out_of_budget.groupby(['Hitpath Offer ID','Scheduling Name','Budget Period Start Date','Revenue-based Budget']).nth(0).reset_index()
out_of_budget

In [ ]:
###############################################################################################################################
#######################               Send email out                                                        ###################
###############################################################################################################################

# create content for email body: 
offer_alert = budget_offer_need_to_swap_1.merge(sms_offer[['Hitpath Offer ID','Scheduling Name']], how= 'left')
email_body = offer_alert.loc[offer_alert['Budget Used %']>= 0.95]
email_body = email_body[['Hitpath Offer ID','Scheduling Name','Cap','Budget','RX Rep','Revenue-based Budget']].value_counts().reset_index(name = 'Number of upcoming drops') 
content = "* Please pay attention to the following offers because the used budget is filled with over 95% of the cap:" + '\n\n'
for i in email_body['RX Rep'].unique(): 
    email_am = email_body[email_body['RX Rep'] == i]
    content += '\t'+i + ":" + '\n'
    for j in range(0,len(email_am)): 
        if email_am.iloc[j]['Revenue-based Budget']==False:
            content+= '\t'+email_am.iloc[j]['Scheduling Name']  + '\n' + '\tTotal Budget: '+ str(email_am.iloc[j]['Budget']) + " " + email_am.iloc[j]['Cap']  +" leads " + '\n' + '\t'+ "There're " + str(email_am.iloc[j]['Number of upcoming drops']) + ' upcoming drops for offer ' + str(email_am.iloc[j]['Hitpath Offer ID']) +'\n'
        else:
            content+= '\t'+email_am.iloc[j]['Scheduling Name']  + '\n' + '\tTotal Budget: '+ str(email_am.iloc[j]['Budget']) + " " + email_am.iloc[j]['Cap']  +" revenue " + '\n' + '\t'+ "There're " + str(email_am.iloc[j]['Number of upcoming drops']) + ' upcoming drops for offer ' + str(email_am.iloc[j]['Hitpath Offer ID']) +'\n'
email_body = offer_alert.loc[(offer_alert['Project Budget Use %']>= 0.95) & (offer_alert['Budget Used %']< 0.95)]
email_body = email_body[['Hitpath Offer ID','Scheduling Name','Cap','Budget','RX Rep','Revenue-based Budget']].value_counts().reset_index(name = 'Number of upcoming drops') 
content += '\n' + "* Please pay attention to the following offers because the projected budget to fill is over 95% of the cap:" + '\n\n'
for i in email_body['RX Rep'].unique(): 
    email_am = email_body[email_body['RX Rep'] == i]
    content += '\t'+i + ":" + '\n' 
    
    for j in range(0,len(email_am)):
        if email_am.iloc[j]['Revenue-based Budget']==False:
            content+= '\t'+email_am.iloc[j]['Scheduling Name'] + '\n' + '\tTotal Budget: '+ str(email_am.iloc[j]['Budget']) + " " + email_am.iloc[j]['Cap']  +" leads " + '\n' + '\t'+ "There're " + str(email_am.iloc[j]['Number of upcoming drops']) + ' upcoming drops for offer ' + str(email_am.iloc[j]['Hitpath Offer ID']) +'\n\n'
        else:
            content+= '\t'+email_am.iloc[j]['Scheduling Name'] + '\n' + '\tTotal Budget: '+ str(email_am.iloc[j]['Budget']) + " " + email_am.iloc[j]['Cap']  +" revenue " + '\n' + '\t'+ "There're " + str(email_am.iloc[j]['Number of upcoming drops']) + ' upcoming drops for offer ' + str(email_am.iloc[j]['Hitpath Offer ID']) +'\n\n'

swap_suggestions1 = swap_suggestions.copy()
all_upcoming_drops1 = all_upcoming_drops.merge(sms_offer[['Hitpath Offer ID','Scheduling Name']], how= 'left')
all_upcoming_drops1 = all_upcoming_drops1.drop(columns=['Projected cumulative budget use','Budget'])
swap_suggestions1['Date'] = swap_suggestions1['Date'].astype(str)
all_upcoming_drops1 = all_upcoming_drops1.merge(swap_suggestions1,how='right',on=['Hitpath Offer ID','DP&Pub', 'Date','RX Rep','Cap','DMA','Revenue-based Budget'])
out_of_budget = all_upcoming_drops1.loc[all_upcoming_drops1['Projected cumulative budget use']>= all_upcoming_drops1['Budget']]
out_of_budget_list = out_of_budget.groupby(['Hitpath Offer ID','Scheduling Name','Budget Period Start Date','Revenue-based Budget']).nth(0).reset_index()

content += '\n' + "* Alert: The below offer(s) are projected to exceed budget caps. Account managers may be reaching out to mailers if swaps are necessary. (Drops format: DP&Pub - Date - Emailer - Analyst)" + '\n'
for i in out_of_budget_list['RX Rep'].unique(): 
    out_of_budget_list_am = out_of_budget_list[out_of_budget_list['RX Rep'] == i]
    content += '\n'+ '\t'+i + ":" 
    for j in range(0,len(out_of_budget_list_am)):
        if out_of_budget_list_am.iloc[j]['Revenue-based Budget']==False:
            content += '\n'+'\t' + out_of_budget_list_am.iloc[j]['Scheduling Name'] + '\n' +'\t'+ out_of_budget_list_am.iloc[j]['Cap'] + " Budget of " + str(out_of_budget_list_am.iloc[j]['Budget']) + " leads started " + out_of_budget_list_am.iloc[j]['Budget Period Start Date'] + " is projected to reach budget cap by " +  out_of_budget_list_am.iloc[j]['Date'] +  '.\n' +'\t'+"Here are swaps we might need to swap: " + "\n"
        else:
            content += '\n'+'\t' + out_of_budget_list_am.iloc[j]['Scheduling Name'] + '\n' +'\t'+ out_of_budget_list_am.iloc[j]['Cap'] + " Budget of " + str(out_of_budget_list_am.iloc[j]['Budget']) + " revenue started " + out_of_budget_list_am.iloc[j]['Budget Period Start Date'] + " is projected to reach budget cap by " +  out_of_budget_list_am.iloc[j]['Date'] +  '.\n' +'\t'+"Here are swaps we might need to swap: " + "\n"
        out_of_budget_for_offer = out_of_budget.loc[out_of_budget['Hitpath Offer ID'] ==  out_of_budget_list_am.iloc[j]['Hitpath Offer ID']] 
        for k in range(0,len(out_of_budget_for_offer)): 
            content += '\t\t'+ str(out_of_budget_for_offer.iloc[k]['DP&Pub']) +' - ' + str(out_of_budget_for_offer.iloc[k]['Date'])  + ' - ' + str(out_of_budget_for_offer.iloc[k]['Emailer']) +   ' - ' + str(out_of_budget_for_offer.iloc[k]['Analyst']) +'\n' 
content += "\n\n Please see attached file for details. \n Have a fanastic day! \n Bests, \n Nathan Mai \n"


notify = ['nathan@rxmg.com', 'lili@rxmg.com', 'g.chao@rxmg.com', 'offernotices@rxmg.com']

for i in notify:
    #send_email(filename,subject,body,toaddr)
    send_email.send_email([filepath], 'SMS Budget Report from Nathan', content, i)